In [71]:
import torch
feature = torch.rand(64, 16, 21, 31)
# print(feature.shape, feature)

In [72]:
flattened = feature.view(-1, 31 * 21)
# print(flattened.shape, flattened)

In [73]:
import torch.nn.functional as F
softmax_attention = F.softmax(flattened/1.0, dim=-1)
# print(softmax_attention.shape, softmax_attention)

In [77]:
import numpy as np
pos_x, pos_y = np.meshgrid(
        np.linspace(-1., 1., 21),
        np.linspace(-1., 1., 31)
        )
pos_x = torch.from_numpy(pos_x.reshape(31*21)).float()
pos_y = torch.from_numpy(pos_y.reshape(31*21)).float()
expected_x = torch.sum(pos_x*softmax_attention, dim=1, keepdim=True)
expected_y = torch.sum(pos_y*softmax_attention, dim=1, keepdim=True)

print(expected_x.shape, expected_y.shape)

torch.Size([1024, 1]) torch.Size([1024, 1])


In [83]:
expected_xy = torch.cat([expected_x, expected_y], 1)
xy = expected_xy
print(expected_xy.shape)

torch.Size([1024, 2])


In [87]:
x = xy[:, 0]
y = xy[:, 1]
rx = torch.reshape(x, (64, 16))
ry = torch.reshape(y, (64, 16))
print(x.shape, rx.shape)

torch.Size([1024]) torch.Size([64, 16])


In [89]:
coord = rx * 320 - 1 + 320 + (ry * 240 + 239) * 640
coord = torch.round(coord).long()
print(coord.shape)

torch.Size([64, 16])


In [98]:
Z = torch.take(flattened_depth, coord)
Z = torch.reshape(Z, (1, 64 * 16))
X = Z / 450 * 640 / 2 * x
Y = Z / 450 * 480 / 2 * y
print(X.shape, Y.shape, Z.shape)
feature_keypoints = torch.cat([X, Y, Z], 0)
print(feature_keypoints.shape)

torch.Size([1, 1024]) torch.Size([1, 1024]) torch.Size([1, 1024])
torch.Size([3, 1024])


In [118]:
delta_xyz = feature_keypoints
target = feature_keypoints + delta_xyz
print(target.shape)
x = target[0, :]
y = target[1, :]
z = target[2, :]
cx = 450 * x * 2 / z / 640
cy = 450 * y * 2 / z / 480
new_xy = torch.cat([cx[None,:], cy[None,:]], 0)
print(new_xy.shape)
print(xy.shape)
grad_input = torch.transpose(new_xy, 1, 0) - xy
print(new_xy.shape, grad_input.shape)

torch.Size([3, 1024])
torch.Size([2, 1024])
torch.Size([1024, 2])
torch.Size([2, 1024]) torch.Size([1024, 2])


In [163]:
data = torch.arange(3072).reshape(64, 48).float()
# data = torch.transpose(torch.arange(3072).reshape(48,64), 1, 0)
# print(data)
size = list(data.shape)
data = torch.transpose(data.reshape(size[0], size[1] // 3, 3), 2, 1)
# print(data)
# data = data.permute(1, 2, 0)
# print(data)
# data = torch.stack((data[:]), 0)
splits_data = torch.split(data, 1, dim = 0)
data = torch.squeeze(torch.cat(splits_data, 2))
# data = data.reshape(0)
print(data.shape, data)
br = torch.rand(3,3)
bt = torch.rand(3,1)
reprojected_pt = torch.matmul(br,data) + bt#[:,None]
# reprojected_pt = reprojected_pt.permute(2, 0, 1)
# reprojected_pt = reprojected_pt.reshape(size[0], size[1])
print(reprojected_pt.shape)

torch.Size([3, 1024]) tensor([[0.0000e+00, 3.0000e+00, 6.0000e+00,  ..., 3.0630e+03, 3.0660e+03,
         3.0690e+03],
        [1.0000e+00, 4.0000e+00, 7.0000e+00,  ..., 3.0640e+03, 3.0670e+03,
         3.0700e+03],
        [2.0000e+00, 5.0000e+00, 8.0000e+00,  ..., 3.0650e+03, 3.0680e+03,
         3.0710e+03]])
torch.Size([3, 1024])


In [120]:
torch.transpose(torch.arange(64).repeat(1,48).reshape(48,64), 1, 0)

tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 2,  2,  2,  ...,  2,  2,  2],
        ...,
        [61, 61, 61,  ..., 61, 61, 61],
        [62, 62, 62,  ..., 62, 62, 62],
        [63, 63, 63,  ..., 63, 63, 63]])

In [78]:
depth = torch.rand(64,1,48,64)
flattened_depth = depth.view(-1, 64 * 48)
print(flattened_depth.shape)

torch.Size([64, 3072])


In [84]:
coord = expected_x * 32 + 31 + (expected_y * 24 + 23) * 64
print(coord.shape, coord)

torch.Size([1024, 1]) tensor([[1521.0413],
        [1496.0164],
        [1504.9271],
        ...,
        [1504.0492],
        [1508.0747],
        [1506.8193]])


In [85]:
rc = torch.reshape(coord, (64, 16))
rc = torch.round(rc).long()
print(rc.shape, rc)

torch.Size([64, 16]) tensor([[1521, 1496, 1505,  ..., 1499, 1489, 1523],
        [1503, 1501, 1496,  ..., 1509, 1492, 1516],
        [1510, 1499, 1506,  ..., 1496, 1492, 1508],
        ...,
        [1488, 1496, 1503,  ..., 1520, 1491, 1511],
        [1506, 1507, 1502,  ..., 1500, 1498, 1495],
        [1510, 1503, 1508,  ..., 1504, 1508, 1507]])


In [81]:
z = torch.take(flattened_depth, rc)
# print(z.shape, z)

In [25]:
# flattened_depth[0, 1478]

tensor(0.1919)

In [26]:
# expected_x

tensor([[ 0.0038],
        [ 0.0033],
        [-0.0053],
        ...,
        [ 0.0034],
        [-0.0050],
        [-0.0101]])

In [37]:
r_expected_x = torch.reshape(expected_x, (1, 64 * 16))
r_expected_y = torch.reshape(expected_y, (1, 64 * 16))
Z = torch.reshape(z, (1, 64 * 16))
# print(r_expected_x.shape, r_expected_x)

torch.Size([1, 1024]) tensor([[ 0.0038,  0.0033, -0.0053,  ...,  0.0034, -0.0050, -0.0101]])


In [38]:
f = 450.0
X = Z / f * r_expected_x
Y = Z / f * r_expected_y
# print(X.shape, Y.shape)

torch.Size([1, 1024]) torch.Size([1, 1024])


In [40]:
output = torch.cat((X,Y,Z), 0)
# print(output.shape)

torch.Size([3, 1024])


In [66]:
# print(output.shape, output)

torch.Size([3, 1024]) tensor([[ 3.0790e-06,  1.8891e-06, -6.9771e-06,  ...,  6.9582e-06,
         -3.2779e-06, -1.2797e-05],
        [-6.6799e-06,  2.0160e-06,  4.5503e-06,  ..., -8.9243e-07,
         -6.2762e-06, -2.8791e-06],
        [ 3.6824e-01,  2.5540e-01,  5.8702e-01,  ...,  9.1723e-01,
          2.9368e-01,  5.6985e-01]])


In [68]:
ct = torch.transpose(output, 1, 0)
# print(ct.shape)
# ct = torch.reshape(ct, (48, 64))
# ct = ct.permute(2, 0, 1)
# print(ct)

torch.Size([1024, 3])
tensor([[ 3.0790e-06, -6.6799e-06,  3.6824e-01],
        [ 1.8891e-06,  2.0160e-06,  2.5540e-01],
        [-6.9771e-06,  4.5503e-06,  5.8702e-01],
        ...,
        [ 6.9582e-06, -8.9243e-07,  9.1723e-01],
        [-3.2779e-06, -6.2762e-06,  2.9368e-01],
        [-1.2797e-05, -2.8791e-06,  5.6985e-01]])


In [70]:
ct = torch.transpose(torch.reshape(torch.reshape(ct, (1, 3072)), (48, 64)), 1, 0)
# print(ct.shape, ct)

torch.Size([64, 48]) tensor([[ 3.0790e-06,  7.3684e-06,  7.0882e-01,  ..., -7.4541e-06,
          6.1931e-06,  2.0213e-02],
        [-6.6799e-06,  2.7796e-01, -1.9122e-06,  ...,  1.4597e-06,
          3.5801e-01, -2.3746e-05],
        [ 3.6824e-01, -1.9344e-05,  1.0800e-06,  ...,  7.0882e-01,
         -6.1404e-06, -2.7265e-05],
        ...,
        [-8.7943e-06,  5.8702e-01, -4.1606e-06,  ...,  1.5940e-05,
          2.0213e-02, -1.2797e-05],
        [ 9.0611e-01,  1.1934e-06, -2.2334e-06,  ...,  8.2196e-01,
         -3.2235e-07, -2.8791e-06],
        [-3.8233e-07,  5.1653e-07,  3.2809e-01,  ..., -6.8265e-07,
         -2.6091e-07,  5.6985e-01]])


In [64]:
output[:,9]

tensor([-9.2672e-06, -1.1165e-05,  4.6974e-01])

In [65]:
ct[29:32, 0]

tensor([ 3.2809e-01, -2.2862e-05, -2.8824e-05])

In [62]:
ct[:, 9, 0]

tensor([-6.4296e-06, -3.5192e-06,  5.6985e-01])

In [54]:
ct[:,0, 0]

tensor([ 3.0790e-06, -6.6799e-06,  3.6824e-01])

In [55]:
x = torch.rand(1,1024)
x

tensor([[0.2493, 0.7042, 0.1844,  ..., 0.5085, 0.2837, 0.9105]])

In [58]:
x = torch.rand(1,1024)
x = torch.reshape(x, (64, 16))
x = torch.transpose(x, 1, 0)
y = torch.rand(1,1024)
y = torch.reshape(y, (64, 16))
y = torch.transpose(y, 1, 0)
z = torch.rand(1,1024)
z = torch.reshape(z, (64, 16))
z = torch.transpose(z, 1, 0)

tensor([[0.2493, 0.7042, 0.1844,  ..., 0.7844, 0.7588, 0.6657],
        [0.8313, 0.1836, 0.2371,  ..., 0.5157, 0.1571, 0.3898],
        [0.3626, 0.2576, 0.6578,  ..., 0.3195, 0.2390, 0.6638],
        ...,
        [0.6438, 0.8383, 0.5530,  ..., 0.3265, 0.3717, 0.3140],
        [0.5593, 0.4217, 0.8428,  ..., 0.6328, 0.2114, 0.0917],
        [0.2612, 0.9464, 0.5104,  ..., 0.5085, 0.2837, 0.9105]])

In [173]:
x = torch.arange(1024)
y = torch.arange(1024,2048)
z = torch.arange(2048, 3072)
print(x.shape,y.shape,z.shape)

torch.Size([1024]) torch.Size([1024]) torch.Size([1024])


In [176]:
x = x.reshape(64,16)
y = y.reshape(64,16)
z = z.reshape(64,16)
xyz = torch.stack([x,y,z], 1).reshape(64,3,1,16)
print(xyz.shape)

torch.Size([64, 3, 1, 16])


In [187]:
dxyz = xyz.clone()
dxyz = torch.squeeze(dxyz)
dxyz = dxyz.permute(1,2,0).transpose(2,1)
dxyz = dxyz.reshape(3,1024)
dxyz

tensor([[   0,    1,    2,  ..., 1021, 1022, 1023],
        [1024, 1025, 1026,  ..., 2045, 2046, 2047],
        [2048, 2049, 2050,  ..., 3069, 3070, 3071]])

In [193]:
output = xyz.clone()
output = torch.squeeze(output).transpose(2,1)
output = output.reshape(64,48)
output[0,:]

tensor([   0, 1024, 2048,    1, 1025, 2049,    2, 1026, 2050,    3, 1027, 2051,
           4, 1028, 2052,    5, 1029, 2053,    6, 1030, 2054,    7, 1031, 2055,
           8, 1032, 2056,    9, 1033, 2057,   10, 1034, 2058,   11, 1035, 2059,
          12, 1036, 2060,   13, 1037, 2061,   14, 1038, 2062,   15, 1039, 2063])

In [196]:
rd = torch.rand(64, 16, 21, 31)
print(rd.shape)
kd = rd.view(-1, rd.shape[1] * rd.shape[2] * rd.shape[3])
print(kd.shape)

torch.Size([64, 16, 21, 31])
torch.Size([64, 10416])


In [198]:
import torch
import torch.nn as nn
from __future__ import print_function, division
import torchvision
from torchvision import datasets, models, transforms

In [201]:
features = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [217]:
modules=list(features.children())
# print(modules)

In [235]:
data = torch.rand(64,4,120,160)
conv1 = nn.Conv2d(4,64,kernel_size=(7,7),stride=(2,2), padding=(3,3),bias=False)
len(modules)
model = nn.Sequential(conv1, *modules[1:7])

In [236]:
output = model(data)
print(output.shape)

torch.Size([64, 256, 8, 10])


In [213]:
modules[0] = nn.Conv2d(4,64,kernel_size=(7,7),stride=(2,2), padding=(3,3),bias=False)

In [219]:
data = torch.rand(64,4,120,160)
model = nn.ModuleList(modules)
output = model(model)

NotImplementedError: 